In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import json
import os
import random
import matplotlib
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

C:\Users\lcruz\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [5]:
def getJsonFile(path):
    '''
    This function loads the data from json file 
    to data frame.
    
    '''
    lines = []
    with open(path, encoding = 'utf-8') as f:
        for i,line in enumerate(f):
            if i < 10000:
                lines.append(json.loads(line))
            else:
                break
    return lines

In [6]:
reviews = pd.DataFrame(getJsonFile('yelp_academic_dataset_review.json'))
business = pd.DataFrame(getJsonFile('yelp_academic_dataset_business.json'))

In [7]:
reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [8]:
business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [9]:
# filter by restaurant and match to review

In [11]:
business_food = business[business['categories'].str.contains('Food')==True].reset_index()

In [12]:
business_food

,index,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
1,4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
2,5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
3,9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
4,11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,9983,EGITUFVLb87GRKapMtZg-w,Dunkin',3820 S Dale Mabry Hwy,Tampa,FL,33611,27.906781,-82.505798,2.0,42,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Donuts, Coffee & Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
2210,9986,GkMeiDDrIlM648Vv1PhkWw,Tropical Smoothie Cafe - Sun City,"3726 Sun City Center Blvd, Ste 103",Sun City,FL,33573,27.713742,-82.372208,3.0,18,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Restaurants, Juice Bars & Smoothies, Food, San...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-21:0', '..."
2211,9989,4iTsgayXGV8zYppognuQXg,Jake's Wayback Burgers,369 W Lancaster Ave,Radnor,PA,19087,40.044662,-75.398588,3.0,18,0,"{'RestaurantsReservations': 'False', 'BikePark...","Food, Restaurants, Burgers","{'Monday': '10:30-20:0', 'Tuesday': '10:30-20:..."
2212,9990,uz3bn1LnXUGPteDoHYhcRQ,KFC,5367 Ehrlich Rd,Tampa,FL,33625,28.083294,-82.542486,1.5,19,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Chicken Shop, Restaurants, Fast Food, Chicken ...","{'Monday': '10:30-23:0', 'Tuesday': '10:30-23:..."


In [13]:
rev_bus_merge = reviews.merge(business, how='inner', on='business_id')

In [14]:
rev_bus_merge

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,name,...,state,postal_code,latitude,longitude,stars_y,review_count,is_open,attributes,categories,hours
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
1,VJxlBnJmCDIy8DFG0kjSow,Iaee7y6zdSB3B-kRCo4z1w,XQfwVwDr-v0ZS3_CbbE5Xw,2.0,0,0,0,This is the second time we tried turning point...,2017-05-13 17:06:55,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
2,S6pQZQocMB1WHMjTRbt77A,ejFxLGqQcWNLdNByJlIhnQ,XQfwVwDr-v0ZS3_CbbE5Xw,4.0,2,0,1,The place is cute and the staff was very frien...,2017-08-08 00:58:18,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
3,WqgTKVqWVHDHjnjEsBvUgg,f7xa0p_1V9lx53iIGN5Sug,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,We came on a Saturday morning after waiting a ...,2017-11-19 02:20:23,Turning Point of North Wales,...,PA,19454,40.210196,-75.223639,3.0,169,1,"{'NoiseLevel': 'u'average'', 'HasTV': 'False',...","Restaurants, Breakfast & Brunch, Food, Juice B...","{'Monday': '7:30-15:0', 'Tuesday': '7:30-15:0'..."
4,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,Zaika,...,PA,19114,40.079848,-75.025080,4.0,181,1,"{'Caters': 'True', 'Ambience': '{'romantic': F...","Halal, Pakistani, Restaurants, Indian","{'Tuesday': '11:0-21:0', 'Wednesday': '11:0-21..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6767,_b2Yvq8LZ9sdNAEjS7jgEw,k9Mvnw4aglONCaS5cBKQnw,yqBU6jO9e988dwxBkstJFQ,5.0,2,0,0,This location has a drive-up window... great f...,2011-02-15 14:44:31,Papa Murphy's,...,IN,46038,39.959788,-86.038480,4.5,7,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Pizza, Restaurants","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'..."
6768,5E6GE3YCCBd5dKVCXWIZgg,RkmOFFHv1vharP_8ARd-Qg,iFzxAYaZYVNr9OLaek1cYg,5.0,0,0,0,I'll admit I messed up and tried to rent a car...,2018-03-15 23:43:33,Enterprise Rent-A-Car,...,PA,19038,40.096043,-75.213632,3.5,11,1,None,"Hotels & Travel, Car Rental","{'Monday': '0:0-0:0', 'Tuesday': '7:30-17:30',..."
6769,HZUfjEGHLihvDZFlurjxpQ,rUs9xSR1LASkh6_grbQlHg,F_KRsGlJJSS7_N2xOErJDw,1.0,2,0,0,I was in the market for a car and went here to...,2011-02-13 21:10:05,Jones West Ford,...,NV,89502,39.489581,-119.788199,1.5,254,0,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Body Shops, Auto Parts & Supplies...","{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'..."
6770,0G8nl_2lbrxqO2ot3NSHxQ,a0REzCBCztw5pS0_BOTa6A,hxsdgbsEQwWUZq91Mx9MCw,3.0,0,0,0,My work booked a private class on a Sunday for...,2015-12-21 03:26:13,Aradia Fitness Edmonton _ Whyte ave,...,AB,T6E 1Z5,53.517929,-113.492202,3.5,11,1,"{'GoodForKids': 'False', 'BikeParking': 'True'...","Dance Studios, Specialty Schools, Pole Dancing...","{'Monday': '18:0-21:0', 'Tuesday': '18:0-21:0'..."


In [ ]:
restaurant = data[data['categories'].str.contains('Restaurant')==True].reset_index()

In [ ]:
restaurant = restaurant.dropna()

In [ ]:
restaurant.to_csv('Yelp Dataset Shortened.csv')

In [16]:
rev_bus_merge['text']

0       If you decide to eat here, just be aware it is...
1       This is the second time we tried turning point...
2       The place is cute and the staff was very frien...
3       We came on a Saturday morning after waiting a ...
4       Wow!  Yummy, different,  delicious.   Our favo...
                              ...                        
6767    This location has a drive-up window... great f...
6768    I'll admit I messed up and tried to rent a car...
6769    I was in the market for a car and went here to...
6770    My work booked a private class on a Sunday for...
6771    Super nice club, much larger than I anticipate...
Name: text, Length: 6772, dtype: object

In [19]:
rev_bus_merge.to_csv('Yelp Dataset Shortened.csv')